In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import Model

In [ ]:
df = pd.read_csv('./../ejerciciosModelosSupervisados/tasa_conversion/datos/KAG_conversion_data.csv')

In [ ]:
# pre-process datos (si es necesario)

df['age'] = df.age.replace({'30-34':1,'35-39':2,'40-44':3,'45-49':4})
df = df.drop(['ad_id','Approved_Conversion','fb_campaign_id'], axis = 1)
df['Spent'] = df.Spent.astype('float')
df['xyz_campaign_id'] = df.xyz_campaign_id.astype(str)
df = pd.get_dummies(df, drop_first=True)

In [ ]:
yvar = df.Total_Conversion
xvars = df[[x for x in df.columns if x != 'Total_Conversion']]

In [ ]:
# dividir train / test para variables independientes (x)  y dependiente (y)
xtrain, xtest, ytrain, ytest = train_test_split(xvars, yvar, train_size = 0.80, random_state = 2)

In [ ]:
# convertir datos en numpy arrays

xtrain = np.array(xtrain)
xtest= np.array(xtest)
ytrain = np.array(ytrain)
ytest = np.array(ytest)

In [ ]:
# arquitectura del modelo utilizando variables - FUNCTIONAL API

# input
model_input = Input(shape=(xtrain.shape[1],))
# primera capa
layer_1 = Dense(256, kernel_regularizer='l2', activation='relu')(model_input)
#segunda capa
layer_2 = Dense(128, kernel_regularizer='l2', activation='relu')(layer_1)
# predicción
model_output = Dense(1)(layer_2)
# entrada y salida
model = Model(inputs=model_input, 
              outputs=model_output)

In [ ]:
error_train_history = []
error_test_history = []

num_epochs = 50

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0002)

mae = tf.keras.losses.MeanAbsoluteError()

for i in range(num_epochs):
    
    print('epoch number: {i}'.format(i=i+1))
    
    with tf.GradientTape() as t:
        
        # aplicar coeficientes a variables y predecir la y
        ytrain_hat = model(xtrain)
        ytest_hat = model(xtest)
        # calcular el error (que buscamos reducir)
        error_train = mae(tf.expand_dims(ytrain, 1), ytrain_hat)
        error_test = mae(tf.expand_dims(ytest, 1), ytest_hat)
        # guardar el error para visualizar luego
        error_train_history.append(error_train.numpy())
        error_test_history.append(error_test.numpy())
    
    # calcular los gradientes de cada uno de los coeficientes con respecto al error    
    gradients = t.gradient(error_train, model.trainable_weights)
    # restar los gradientes * learning_rate a cada uno de los coeficientes
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))     
    
    if i > 10:
        
        if error_train_history[-4] < error_train_history[-1]:
            
            break
            
            print('stopping training as no improvement in 3 epochs')
    
    print('train error: {error_train}, test error: {error_test}'.format(error_train=error_train,error_test=error_test))

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))
ax.plot(range(len(error_train_history)), error_train_history, label='train loss')
ax.plot(range(len(error_test_history)), error_test_history, label='train loss')
ax.legend()
plt.ylabel("loss")
plt.title('Error at Training')
plt.show()